# <center> <img src='../images/fsktm.jpg' width="500" height="400"> </center>
# <center> WQD7002 - SCIENCE DATA RESEARCH PROJECT </center>
## <center> NEURAL RUBIK’S </center>
## <center> Solving Rubik's Cube Using Nueral Network (Hueristic Learning) </center>
### <center> Scripted by : Gunasegarran Magadevan (WQD170002) </center>
### <center> Supervised by : Dr.Aznul Qalid Md Sabri </center>
# <center> <img src="../images/RubiksNeural.jpg" width="500" height="600"> </center>
----



### STEP 1 : Installing and upgrading the package

In [1]:
# Upgrading the pip package to the latest version
!python -m pip install PyHamcrest --upgrade --quiet
!python -m pip install tensorflow --upgrade --quiet
!python -m pip install rubikai --no-cache-dir --upgrade --quiet
!python -m pip install seaborn --no-cache-dir --upgrade --quiet
!python -m pip install keras --upgrade --quiet
!python -m pip install numpy --upgrade --quiet
!pip install -U -q PyDrive

# Tensorflow package manually through terminal or cmd - https://anaconda.org/conda-forge
#conda install -c conda-forge tensorflow
#conda install -c conda-forge numpy

-ensorflow 1.11.0 has requirement tensorboard<1.12.0,>=1.11.0, but you'll have tensorboard 1.13.1 which is incompatible.


### STEP 2 : Importing the package

In [2]:
import rubikai as rubik
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import matplotlib.pyplot as plt

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'google.colab'

### STEP 3 : Import/Export Using Google Drive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
def refresh_gdrive_token():
  global auth
  global drive
  global gauth
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

In [ ]:
SAVE_TO_DRIVE = True

def upload_file_to_drive(local_filename, remote_filename=None):
  refresh_gdrive_token()
  if remote_filename is None:
    remote_filename = 'drive_' + local_filename
  uploaded = drive.CreateFile({'title': remote_filename})
  uploaded.SetContentFile(local_filename)
  uploaded.Upload()
  return


def download_model_from_drive(remote_filename, local_filename=None):
  refresh_gdrive_token()
  if local_filename is None:
    local_filename = 'local_' + remote_filename
  file_list = drive.ListFile(
      {'q': "title = '{}'".format(remote_filename)}).GetList()
  if file_list:
    file_list[0].GetContentFile(local_filename)
    model = keras.models.load_model(local_filename)
    return model
  else:
    print('file not found in drive')

In [ ]:
# save/load model functions
def save_model(model, filename):
  global SAVE_TO_DRIVE
  if SAVE_TO_DRIVE is True:
    model.save(filename)
    upload_file_to_drive(filename, filename)
    return


def load_model(filename):
  return download_model_from_drive(filename, filename)

### STEP 4 : Helper Functions

Below are functions related to probability vectors. 
* `create_prob_vector` generates the "real" distribution of cube configurations
* `convex_combination` returns a convex combination of two vectors
* `convex_combination_probabilities` returns a convex combination of the real distribution and the uniform one

In [ ]:
# Number of states in each distance from goal (taken from http://cube20.org/qtm/)
count_vector = np.array([1, 12, 114, 1068, 10011, 93840, 878880, 8221632,
                         76843595, 717789576, 6701836858, 62549615248,
                         583570100997, 5442351625028, 50729620202582,
                         472495678811004, 4393570406220123, 40648181519827392,
                         368071526203620348, 3e18, 14e18, 19e18, 7e18, 24e15,
                         150000, 36, 3])

def create_prob_vector(lower, upper):
  vec = count_vector[lower:upper]
  return vec / np.sum(vec)


def convex_combination(v , u, alpha):
  """ return covex combination of u, v i.e v*alpha + u*(1- alpha) """
  assert len(u) == len(v), 'u ,v must have same length'
  assert 0 <= alpha <= 1, 'alpha must be between 0 and 1'
  return np.array(v)*alpha + np.array(u)*(1-alpha)

def convex_combination_probabilities(alpha, lower, upper):
  """ 
  returns convex combination of real probability and uniform distribution
  real_probabilites*alpha + uniform*(1- alpha)
  """
  rel_prob = create_prob_vector(lower, upper)
  n = len(rel_prob)
  uniform = np.ones(n) / n
  return convex_combination(rel_prob, uniform, alpha)

### STEP 5 :  Functions for Learning

In [ ]:
def get_features_from_cube(cube):
  """ transforms the cube's array to 1d binary array """
  binary_array = keras.utils.to_categorical(cube.to_array(), rubik.NUM_FACES)
  return binary_array.flatten()

 
def data_generator(cube_layers, max_d, batch_size, p=None):
  """
  generates batches of scrambled cubes data, coupled with the number
  of scramble moves per row
  """
  new_dim = len(get_features_from_cube(rubik.Cube(cube_layers)))
  while True:
    data = np.empty((batch_size, new_dim), dtype=np.int8)
    labels = np.empty(batch_size, dtype=np.int8)
    for i in range(batch_size):
      c = rubik.Cube(cube_layers)
      d = np.random.choice(np.arange(max_d+1), p=p)
      rand_seq = rubik.generate_random_sequence(cube_layers, d)
      c.apply(rand_seq)
      data[i, :] = get_features_from_cube(c)
      labels[i] = d
    yield data, labels  


def create_dnnregressor(cube_layers, hidden_units, dropout=None,
                        optimizer='adagrad', loss='mse'):
  """
  creates a fully connected multi-layer perceptron with non-linear activations
  to perform regression.
  
  :param cube_layers: the number of cube layers this model should operate on
  :param hidden_units: list of integers specifying how many hidden neurons
                       are in each layer
  :param dropout: if None, no dropout is used. if a single integer, uses this
                  dropout rate after each layer. otherwise, should be a list of
                  integers the same length as hidden_units specifying dropout 
                  rate after each layer
  :param optimizer: which (keras) optimizer to use
  :param loss: which (keras) loss to use
  :returns: a compiled keras.Sequential model
  """
  # input checks
  assert hasattr(hidden_units, '__len__'), 'hidden_units must be array-like'
  assert len(hidden_units) > 0, 'hidden_units cannot be empty'
  if dropout is not None:
    if not hasattr(dropout, '__len__'):
      dropout = [dropout] * len(hidden_units)
    else:
      assert len(hidden_units) == len(dropout)
  # define some constant model parameters
  activation = 'relu'
  out_activation = 'relu'
  input_dim = len(get_features_from_cube(rubik.Cube(cube_layers)))
  
  # create a sequential model and add the first layer
  model = keras.Sequential()
  model.add(keras.layers.Dense(hidden_units[0],
                               input_dim=input_dim,
                               activation=activation))
  if dropout is not None:
    model.add(keras.layers.Dropout(dropout[0]))
    
  # add the rest of the hidden layers
  for i in range(1, len(hidden_units)):
    model.add(keras.layers.Dense(hidden_units[i], activation=activation))
    if dropout is not None:
      model.add(keras.layers.Dropout(dropout[i]))

  # define the output layer
  model.add(keras.layers.Dense(1, activation=out_activation))
  model.compile(optimizer=optimizer, loss=loss)
  return model


def learn_heuristic(layers, max_d, p, steps, epochs, batch_size, hidden_units,
                    dropout=None, optimizer='adagrad', loss='mse'):
  """
  trains a model with the given config.
  
  :param layers: number of cube layers
  :param max_d: maximum number of scramble steps
  :param p: a probability distribution according to which the
            scramble steps number is chosen (array of length max_d+1)
            (for uniform dist. use None)
  :param steps: number of training steps per epoch
  :param epochs: number of epochs
  :param batch_size: number of cube instances in each training step
  :param hidden_units: number of dnn layers and neurons in each layer
                       (an array of integers)
  :param dropout: same as in create_dnnregressor
  :param optimizer: same as in create_dnnregressor
  :param loss: same as in create_dnnregressor
  :returns: a pair (estimator, history), where estimator is a (trained)
            keras model, and history is the training Keras history object
  """
  # set up parameters
  c = rubik.Cube(layers)
  # initialize the model
  estimator = create_dnnregressor(
      cube_layers=layers,
      hidden_units=hidden_units,
      dropout=dropout,
      optimizer=optimizer,
      loss=loss
  )
  # train the model
  history = estimator.fit_generator(
      data_generator(layers, max_d, batch_size, p), steps, epochs
  )
  return estimator, history


def model_to_heuristic(model):
  """ creates a heuristic based on the given keras model """
  
  def _model_h(cube, problem=None):
    features = get_features_from_cube(cube)
    return model.predict(np.reshape(features, (1, -1)))[0][0]
  
  return _model_h

### STEP 6 :  Functions for Training

#### $3\times3\times3$
We try different network architectures and see which one performs best.

The heuristics are then save in the following format:

`<layers>_<max_d>_<hidden_1>_..._<hidden_k>.h5`

where `hidden_i` is the number of neurons in the `i`'th hidden layer, and `layers` is the number of layers in the cube, and `max_d` is the maximal number of scramble moves.

For example, a $3\times 3 \times 3$ model with 3 layers of 50 neurons each, and `max_d=8` is saved as: 
"`3_8_50_50_50.h5`".

In [ ]:
def get_model_filename(layers, max_d, hidden_units):
  delim = '_'
  suffix = '.h5'
  return delim.join([str(layers), str(max_d)] + 
                    [str(h) for h in hidden_units]) + suffix

### $Definations$
* `max_d` (maximal number of scramble moves)
* `p` (distribution for the number of moves): $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$, where $\vec \rho$ is the real distribution of cube configurations, and $\vec u$ is the unfirom distribution
* `steps` (training steps per epoch)
* `epochs` (number of epochs)
* `batch_size` (number of examples per training step)


In [ ]:
layers = 3
max_d = 25
p = convex_combination_probabilities(0.1, 0, max_d+1)
steps = 100
epochs = 100
batch_size = 5
dropout = None

### $\hat h_1$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  3 hidden layers with 100, 90, 80 neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_1 = [100, 90, 80]
m1, history1 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_1, dropout)
save_model(m1, get_model_filename(layers, max_d, hidden_units_1))

### $\hat h_2$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$ (same as in $\hat h_1$)
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  4 hidden layers with 100, 90, 80, 70 neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_2 = [100, 90, 80, 70]
m2, history2 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_2, dropout)
save_model(m2, get_model_filename(layers, max_d, hidden_units_2))

### $\hat h_3$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$ (same as in $\hat h_1$)
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  5 hidden layers with 100, 90, 80, 70, 60 neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_3 = [100, 90, 80, 70, 60]
m3, history3 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_3, dropout)
save_model(m3, get_model_filename(layers, max_d, hidden_units_3))

### $\hat h_4$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$ (same as in $\hat h_1$)
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  6 hidden layers with 10, 90, 80, 70, 60, 50 neurons neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_4 = [100, 90, 80, 70, 60, 50]
m4, history4 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_4, dropout)
save_model(m4, get_model_filename(layers, max_d, hidden_units_4))

### $\hat h_5$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$ (same as in $\hat h_1$)
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  7 hidden layers with 100, 90, 80, 70, 60, 50, 40 neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_5 = [100, 90, 80, 70, 60, 50, 40]
m5, history5 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_5, dropout)
save_model(m5, get_model_filename(layers, max_d, hidden_units_5))

### $\hat h_6$
* `max_d`: `10`
* `p`: $0.1 \cdot \vec \rho + 0.9 \cdot \vec u$ (same as in $\hat h_1$)
* `steps`: `100`
* `epochs`: `100`
* `batch_size`: `8`
* Net architecture: 
  *  8 hidden layers with 100, 90, 80, 70, 60, 50, 40, 30 neurons
  * default ReLU activations
  * No edge dropout

In [ ]:
hidden_units_6 = [100, 90, 80, 70, 60, 50, 40, 30]
m6, history6 = learn_heuristic(layers, max_d, p, steps, epochs,
                               batch_size, hidden_units_6, dropout)
save_model(m6, get_model_filename(layers, max_d, hidden_units_6))